**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
!pip install keras-flops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 36.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 44.5 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
dask-cudf 21.10.1 requires cupy-cuda114, which is not install

In [7]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import np_utils

In [8]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [9]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [10]:
# Преобразуйте метки классов в one_hot формат
y_train = np_utils.to_categorical(y_train, CLASSES)
y_val = np_utils.to_categorical(y_val, CLASSES)

In [11]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [143]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(filters=18, kernel_size=4, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.AveragePooling2D(pool_size=(3, 3), strides=2, padding='same'),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [144]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-25 18:31:18.444389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:31:18.444751: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-25 18:31:18.444862: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-25 18:31:18.445423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:31:18.445890: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:31:18.446224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

In [145]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 16, 16, 18)        882       
_________________________________________________________________
batch_normalization_53 (Batc (None, 16, 16, 18)        72        
_________________________________________________________________
activation_89 (Activation)   (None, 16, 16, 18)        0         
_________________________________________________________________
average_pooling2d_36 (Averag (None, 8, 8, 18)          0         
_________________________________________________________________
flatten_37 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_36 (Dense)             (None, 100)               115300    
_________________________________________________________________
activation_90 (Activation)   (None, 100)             

In [146]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [147]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=256
)

Epoch 1/256
391/391 [==============================] - 2s 4ms/step - loss: 3.5262 - accuracy: 0.2029 - val_loss: 3.3037 - val_accuracy: 0.2391
Epoch 2/256
391/391 [==============================] - 1s 3ms/step - loss: 2.9241 - accuracy: 0.3033 - val_loss: 3.0952 - val_accuracy: 0.2733
Epoch 3/256
391/391 [==============================] - 1s 3ms/step - loss: 2.6778 - accuracy: 0.3532 - val_loss: 2.9568 - val_accuracy: 0.2980
Epoch 4/256
391/391 [==============================] - 1s 3ms/step - loss: 2.5051 - accuracy: 0.3886 - val_loss: 2.8422 - val_accuracy: 0.3275
Epoch 5/256
391/391 [==============================] - 1s 3ms/step - loss: 2.3771 - accuracy: 0.4132 - val_loss: 2.7855 - val_accuracy: 0.3388
Epoch 6/256
391/391 [==============================] - 1s 3ms/step - loss: 2.2745 - accuracy: 0.4354 - val_loss: 2.7771 - val_accuracy: 0.3435
Epoch 7/256
391/391 [==============================] - 1s 3ms/step - loss: 2.1946 - accuracy: 0.4544 - val_loss: 2.7819 - val_accuracy: 0.3499